In [56]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("traffic").getOrCreate()
from pyspark.sql.functions import * 

In [57]:
df = spark.read.csv("From100G.csv",header=True, inferSchema=True)

In [58]:
#df.printSchema()

In [59]:
revised_df = df.select(["tpep_picku","id","pickup_lon","pickup_lat"]).orderBy("tpep_picku")
#revised_df.toPandas().to_csv('Traffic_PickUp.csv')

In [60]:
revised_df = revised_df.select(hour(df["tpep_picku"]).alias('hours'),"id","pickup_lon")
#revised_df.select(countDistinct("id")).show()

In [61]:
revised_df.show()

+-----+---+-------------+
|hours| id|   pickup_lon|
+-----+---+-------------+
|   10|309|-73.952445984|
|   10|381|-73.974319458|
|   10|254|-73.988838196|
|   10|309|-73.954147339|
|   10|454|-73.988494873|
|   10|454|-73.986076355|
|   10|329| -73.98551178|
|   10|430|-73.978416443|
|   10|402| -74.00579071|
|   10|258|-73.956924438|
|   10|378|-73.994003296|
|   10|380|-73.977714539|
|   10|551|-74.007522583|
|   10|329|-73.985588074|
|   10|260|-73.948204041|
|   10|308|-73.963226318|
|   10|356|-73.977333069|
|   10|309| -73.95426178|
|   10|284|-73.950744629|
|   10|205| -73.97833252|
+-----+---+-------------+
only showing top 20 rows



In [62]:
#new_df = revised_df.select("id").orderBy('id').show()
#new_df.count()

In [63]:
for i in range(10,12):
    Time = revised_df.filter(revised_df["hours"] == i)
    hours = Time.groupBy("id").count().orderBy("id").withColumnRenamed("count","count@%s"%(i)).show()
    #hours.toPandas().to_csv('mycsv.csv')

+---+--------+
| id|count@10|
+---+--------+
|  0|     124|
| 12|       1|
| 34|       2|
| 59|       1|
| 83|       3|
| 84|       1|
| 85|       1|
|107|       4|
|108|       3|
|111|       1|
|112|       1|
|132|      14|
|133|       3|
|134|       2|
|135|       1|
|156|      12|
|157|      13|
|158|       6|
|159|       1|
|160|       2|
+---+--------+
only showing top 20 rows

+---+--------+
| id|count@11|
+---+--------+
|  0|     552|
| 10|       9|
| 34|       3|
| 38|       1|
| 41|       1|
| 58|       1|
| 59|       9|
| 60|       1|
| 61|       5|
| 83|      10|
| 85|       3|
| 86|       4|
| 87|       3|
|107|      11|
|108|      28|
|109|       3|
|110|       6|
|111|       4|
|112|       1|
|113|       1|
+---+--------+
only showing top 20 rows

